In [1]:
(*
 * Simple arithmetic language
 *)
type exp = 
  | Val of int
  | Plus of exp * exp
  | Div of exp * exp

type exp = Val of int | Plus of exp * exp | Div of exp * exp


In [4]:
let my_exp1 = Plus (Div (Val 4, Val 2), Val 6)

val my_exp1 : exp = Plus (Div (Val 4, Val 2), Val 6)


In [6]:
let rec eval (e:exp) : int = match e with
  | Val n -> n
  | Plus(e1,e2) ->
    let v1 = eval e1 in
    let v2 = eval e2 in
    v1+v2
  | Div (e1,e2) ->
    let v1 = eval e1 in
    let v2 = eval e2 in
    v1/v2

val eval : exp -> int = <fun>


In [7]:
eval my_exp1

- : int = 8


In [8]:
let my_exp2 = Plus (Div (Val 4, Val 0), Val 6)

val my_exp2 : exp = Plus (Div (Val 4, Val 0), Val 6)


In [9]:
eval my_exp2

error: runtime_error

In [10]:
let rec eval (e:exp) : int option = match e with
  | Val n -> Some n
  | Plus(e1,e2) ->
    let v1 = eval e1 in
    begin match v1 with
      | None -> None
      | Some n1 ->
        let v2 = eval e2 in
        begin match v2 with
          | None -> None
          | Some n2 -> Some (n1+n2)
        end
    end
  | Div (e1,e2) ->
    let v1 = eval e1 in
    begin match v1 with
      | None -> None
      | Some n1 ->
        let v2 = eval e2 in
        begin match v2 with
          | None -> None
          | Some 0 -> None
          | Some n2 -> Some (n1/n2)
        end
    end

val eval : exp -> int option = <fun>


In [11]:
eval my_exp2

- : int option = None


In [13]:
(*
 * For any 2 computations f and g:
 * 1. Do computation f.
 * 2. If result of f is None then return None.
 * 3. If the result of f is Some v, then execute (g v)
 *)
 let bind (v : int option) (g: int -> int option) : int option = 
   match v with
     | Some v1 -> g v1
     | None -> None
 
let return (v:int): int option = Some v 

val bind : int option -> (int -> int option) -> int option = <fun>


val return : int -> int option = <fun>


In [14]:
let rec eval (e:exp) : int option = match e with
  | Val n -> Some n
  | Plus(e1,e2) -> 
      bind (eval e1) 
           (fun v1 -> bind (eval e2) 
                           (fun v2 -> return (v1+v2))) 
  | Div (e1,e2) ->
      bind (eval e1) 
           (fun v1 -> bind (eval e2) 
                           (fun v2 -> if v2 = 0 then None
                                      else return (v1/v2)))

val eval : exp -> int option = <fun>


In [15]:
eval my_exp2

- : int option = None


In [16]:
let (>>=) = bind

val ( >>= ) : int option -> (int -> int option) -> int option = <fun>


In [17]:
let rec eval (e:exp) : int option = match e with
  | Val n -> Some n
  | Plus(e1,e2) -> 
      eval e1 >>= fun v1 -> 
      eval e2 >>= fun v2 -> 
      return (v1+v2)
  | Div (e1,e2) ->
      eval e1 >>= fun v1 -> 
      eval e2 >>= fun v2 ->
      if v2 = 0 then None else return (v1/v2)

val eval : exp -> int option = <fun>


In [19]:
let (let*) = bind

val ( let* ) : int option -> (int -> int option) -> int option = <fun>


In [20]:
let rec eval (e:exp) : int option = match e with
  | Val n -> Some n
  | Plus(e1,e2) -> 
      let* v1 = eval e1 in
      let* v2 = eval e2 in
      return (v1+v2)
  | Div (e1,e2) ->
      let* v1 = eval e1 in
      let* v2 = eval e2 in
      if v2 = 0 then None else return (v1/v2)

val eval : exp -> int option = <fun>


In [22]:
let rec eval_exn (e:exp) : int = match e with
  | Val n -> n
  | Plus(e1,e2) ->
    let v1 = eval_exn e1 in
    let v2 = eval_exn e2 in
    v1+v2
  | Div (e1,e2) ->
    let v1 = eval_exn e1 in
    let v2 = eval_exn e2 in
    v1/v2

val eval_exn : exp -> int = <fun>


In [25]:
type 'a t = 'a option
let return (v:'a) : 'a t = Some v
let bind (v:'a option) (f : 'a -> 'a option) : 'a option = 
  match v with
  | None -> None
  | Some v -> f v

type 'a t = 'a option


val return : 'a -> 'a t = <fun>


val bind : 'a option -> ('a -> 'a option) -> 'a option = <fun>


In [27]:
module type MONAD = sig
type 'a t
val return: 'a -> 'a t
val bind: 'a t -> ('a -> 'a t) -> 'a t
end

module type MONAD =
  sig
    type 'a t
    val return : 'a -> 'a t
    val bind : 'a t -> ('a -> 'a t) -> 'a t
  end


In [30]:
let list_return (v: 'a) : 'a list = [v]
(* val list_bind : 'a list -> ('a -> 'a list) -> 'a list *)
let list_bind (l: 'a list) (f: 'a -> 'a list) : 'a list = 
  List.concat (List.map f l)

val list_return : 'a -> 'a list = <fun>


val list_bind : 'a list -> ('a -> 'a list) -> 'a list = <fun>


### Monad Laws


1. Left Identity: `(return v) >>= f` = `f v` 
2. Right Identity: `f >>= return = f`
3. Associativity: `(m >>= f) >>= g = m >>= (fun x -> f x >= g)`

In [32]:
(return 2) >>= f
=
bind (return 2) f
=
match return 2 with
 | None -> None
 | Some 2 -> f 2
=
match Some 2 with
 | None -> None
 | Some 2 -> f 2
=
f 2

error: compile_error

In [ ]:
(*
 * In OCaml
 *)

  let * v1 = c1 in
  let * v2 = c2 v1 in
  ...
  return vn
}